# TRABAJO PRÁCTICO FINAL INTEGRADOR

Este proyecto es un notebook de Google Colab que permite realizar web scraping de artículos de noticias de Infobae y La Nación, unificarlos en un archivo CSV y aplicar diversas técnicas de procesamiento de lenguaje natural (NLP) y análisis de datos. Finalmente, se crea una interfaz interactiva utilizando Gradio para visualizar y analizar el contenido extraído y procesado.

## Características:

- **Web Scraping:** Extrae títulos y texto de los últimos artículos de Infobae y La Nación.
- **Unificación de Datos:** Combina los datos de ambos sitios en un único DataFrame y lo exporta a un archivo CSV.
- **Procesamiento de Lenguaje Natural (NLP):**
    - **Limpieza y Tokenización:** Preprocesamiento del texto para eliminar ruido y dividirlo en unidades significativas.
    - **Generación de Nubes de Palabras (WordCloud):** Visualización de las palabras más frecuentes en el texto.
    - **Extracción de Entidades Nombradas (NER):** Identificación y clasificación de personas, lugares y organizaciones mencionadas en los artículos utilizando Gemini.
    - **Análisis de Sentimiento:** Determinación de la polaridad (positivo, negativo, neutro) del texto utilizando TextBlob y visualización de la distribución general de sentimientos.
    - **Generación de Resúmenes y Tweets:** Creación de resúmenes concisos y tweets con modismos argentinos utilizando Gemini.
- **Interfaz Interactiva con Gradio:** Permite seleccionar artículos, visualizar su texto, aplicar funciones de NLP y ver los resultados de forma interactiva.

## Cómo usar:

1. **Abrir el Notebook:** Abre el notebook en Google Colab.
2. **Ejecutar las Celdas:** Ejecuta secuencialmente todas las celdas del notebook. Esto instalará las dependencias, realizará el web scraping, procesará los datos y lanzará la interfaz Gradio.
3. **Interactuar con la Interfaz Gradio:** Una vez que la interfaz Gradio se haya lanzado (aparecerá un enlace público), ábrela en tu navegador.
    - Selecciona un artículo del menú desplegable.
    - Utiliza los botones para:
        - "Mostrar texto": Ver el texto completo del artículo.
        - "Limpiar texto": Ver el texto después de la limpieza y tokenización.
        - "Generar WordCloud": Visualizar la nube de palabras.
        - "Extraer Entidades (NER)": Ver las entidades nombradas extraídas.
        - "Análisis de Sentimiento": Ver el sentimiento del conjunto de artículos y la distribución general de sentimientos.
        - "Resumen": Obtener un resumen del artículo generado por Gemini.
        - "Generador de Tweet": Obtener un tweet generado por Gemini basado en el artículo.

## Requisitos:

- Cuenta de Google para acceder a Google Colab.
- Clave de API de Google Gemini (configurada en los secretos de Colab como `GOOGLE_API_KEY`).

## Dependencias:

Las dependencias se instalan automáticamente al ejecutar la primera celda del notebook (`!pip install...`). Incluyen:

- `gradio`
- `transformers`
- `sentencepiece`
- `spacy`
- `wordcloud`
- `matplotlib`
- `textblob`
- `requests`
- `beautifulsoup4`
- `pandas`
- `google-generativeai`
- `nltk`

## Archivos generados:

- `datos_combinados.csv`: Archivo CSV que contiene los artículos scrapeados de Infobae y La Nación.

#1. Instalación de librerías y módulos necesarios.

In [28]:
%%capture
!pip install gradio transformers sentencepiece spacy wordcloud matplotlib textblob
!python -m spacy download es_core_news_sm

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import uniform
from google.colab import userdata
import os
from google import genai
from google.genai import types
import pandas as pd
import gradio as gr
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from collections import Counter
from textblob import TextBlob
from transformers import AutoTokenizer, AutoModelWithLMHead, pipeline
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# 2. Web Scraping de Infobae y La Nación

In [30]:
## WEB SCRAPING INFOBAE ##

# URL base
BASE_URL = "https://www.infobae.com"

# URL que contiene los links de artículos (puedes cambiar esta a otra categoría o portada)
page_url = "https://www.infobae.com/ultimas-noticias/"

# Obtener el HTML
response = requests.get(page_url)
soup = BeautifulSoup(response.content, "html.parser")

# Extraer todos los <a> con clase "story-card-ctn"
links = soup.find_all("a", class_="feed-list-card")

# Extraer href y guardar en lista
hrefs = [link.get("href") for link in links if link.get("href")]

# Limitar a los primeros 20
hrefs = hrefs[:20]

# Para almacenar los resultados
data = []

# Recorrer cada href
for href in hrefs:
    full_url = BASE_URL + href if href.startswith("/") else href
    try:
        # Pequeña pausa para no sobrecargar el servidor
        sleep(uniform(1, 3))

        article_response = requests.get(full_url)
        article_soup = BeautifulSoup(article_response.content, "html.parser")

        # Extraer el título del artículo
        h1_element = article_soup.find('h1')
        id_value = h1_element.get('id', '')
        titulo = id_value.replace('-', ' ')

        # Extraer todos los <p> y juntar su texto
        paragraphs = article_soup.find_all("p")
        texto = " ".join([p.get_text(strip=False) for p in paragraphs])

        # Guardar en data
        data.append(["infobae", titulo, texto])

    except Exception as e:
        print(f"Error procesando {full_url}: {e}")
        continue

# Crear DataFrame
df_infobae = pd.DataFrame(data, columns=["fuente", "titulo", "texto"])

In [31]:
# --- WEB SCRAPING LA NACION ---

# URL base
BASE_URL = "https://www.lanacion.com.ar"
page_url = "https://www.lanacion.com.ar/ultimas-noticias/"

# Obtener HTML de la página principal
response = requests.get(page_url)
soup = BeautifulSoup(response.content, "html.parser")

# Extraer todos los <a> con href (sin clase específica)
links = soup.select("h2 a[href]")

# Filtrar hrefs que parezcan links válidos de artículos (opcional: que contengan alguna palabra clave o estructura)
hrefs = [link['href'] for link in links if link['href'].startswith('/')]

# Evitar duplicados
hrefs = list(set(hrefs))

# Limitar a los primeros 20
hrefs = hrefs[:20]

# Lista para almacenar los datos
data = []

# Recorrer los hrefs y scrapear cada artículo
for href in hrefs:
    full_url = BASE_URL + href
    try:
        sleep(uniform(1, 3))  # Pausa para evitar bloqueo

        article_response = requests.get(full_url)
        article_soup = BeautifulSoup(article_response.content, "html.parser")

        # Extraer el título del artículo
        h1_element = article_soup.find('h1', class_="com-title")
        titulo = "".join(h1_element.get_text(strip=False))

        # Extraer todo el texto de los <p>
        paragraphs = article_soup.find_all("p", class_="com-paragraph")
        texto = " ".join([p.get_text(strip=False) for p in paragraphs])

        # Agregar a la lista de resultados
        data.append(["lanacion", titulo, texto])

    except Exception as e:
        print(f"Error al procesar {full_url}: {e}")
        continue

# Crear un DataFrame con los resultados
df_lanacion = pd.DataFrame(data, columns=["fuente", "titulo", "texto"])

df_lanacion.to_csv("lanacion.csv", index=False)

In [32]:
# Unificamos los df de infobae y la nacion en un unico df que exportamos


df = pd.concat([df_infobae, df_lanacion], ignore_index=True)

# Guardar en un CSV
df.to_csv("datos_combinados.csv", index=False)

print("✅ CSV guardado como 'datos_combinados.csv'")

✅ CSV guardado como 'datos_combinados.csv'


# 3. Funciones

In [33]:
# Cargar el modelo de Gemini

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
cliente = genai.Client(api_key=GOOGLE_API_KEY)
MODEL_ID = "gemini-2.5-flash-preview-05-20"

# Cargar modelo spaCy español
nlp = spacy.load("es_core_news_sm")

nltk.download('punkt')  # Para la tokenización de palabras
nltk.download('stopwords')  # Para las palabras comunes

# Cargar CSV
df = pd.read_csv("datos_combinados.csv")

# Opciones para dropdown
opciones = df["titulo"].dropna().astype(str).unique().tolist()

# Cargar el modelo de resumen de Hugging Face (T5 o BART para español)
# summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
# summarizer = pipeline("summarization", model="josmunpen/mt5-small-spanish-summarization")
# summarizer = pipeline("summarization", model="t5-small", tokenizer="t5-small")

# Función para mostrar texto
def mostrar_texto(href_seleccionado):
    fila = df[df["titulo"].astype(str) == str(href_seleccionado)]
    if fila.empty:
        return "⚠️ No se encontró el artículo."
    texto = fila.iloc[0]["texto"]
    if not texto or texto.strip() == "":
        return "⚠️ Artículo sin contenido válido."
    return texto

# 1. Funciones de limpieza y wordcloud

# Función de Limpieza
def limpiar_texto(texto):
    texto = texto[11:]
    texto = re.sub(r"[^\w\s]", "", texto)  # Quitar puntuación
    texto = re.sub(r"\d+", "", texto)      # Quitar números
    texto = texto.lower()       # Normalizar y pasar a minúsculas
    return texto.strip()


# Preprocesamiento NLP
def procesar_texto(texto):
    texto_limpio = limpiar_texto(texto)
    doc = nlp(texto_limpio)
    tokens = [t.lemma_ for t in doc if t.is_alpha and not t.is_stop]
    return " ".join(tokens)


# Generar WordCloud
def generar_wordcloud(texto):
    if not texto or not isinstance(texto, str) or texto.strip() == "":
        print("⚠️ Texto vacío o inválido.")
        return None
    try:
        doc = nlp(texto)
        tokens = [token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop and len(token.text) > 2]
        if not tokens:
            print("⚠️ No hay tokens válidos después de filtrar.")
            return None
        texto_procesado = " ".join(tokens)
        wordcloud = WordCloud(width=800, height=400, max_words=100, background_color="white").generate(texto_procesado)
        buf = BytesIO()
        plt.figure(figsize=(10,5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.tight_layout(pad=0)
        plt.savefig(buf, format='png')
        plt.close()
        buf.seek(0)
        img = Image.open(buf)
        return img
    except Exception as e:
        print(f"❌ Error generando WordCloud: {e}")
        return None


# 2. Función para NER: extraer y contar entidades en texto seleccionado

def realizar_ner(texto):
    pregunta = f"""
    Extraé todas las entidades nombradas del siguiente texto en español argentino y clasificalas, solo necesito la respuesta:

    CATEGORÍAS:
    - PERSONA: Nombres de personas
    - LUGAR: Ciudades, países, barrios, direcciones, lugares específicos
    - ORGANIZACIÓN: Empresas, universidades, instituciones

    FORMATO DE RESPUESTA:
    [ENTIDAD] → [CATEGORÍA] → [BREVE EXPLICACIÓN]

    TEXTO A ANALIZAR:
    {texto}
    """
    respuesta = cliente.models.generate_content(
                model=MODEL_ID,
                contents=[pregunta] # Pasa la pregunta como contenido
                    )
    return respuesta.text



# 3. Funciones para análisis de sentimiento

# Función auxiliar para clasificar polaridad con TextBlob
def clasificar_sentimiento(texto):
    if not texto or texto.strip() == "":
        return "Neutro"
    try:
        blob = TextBlob(texto)
        pol = blob.sentiment.polarity
        if pol > 0.1:
            return "Positivo"
        elif pol < -0.1:
            return "Negativo"
        else:
            return "Neutro"
    except:
        return "Neutro"

# Calcular sentimiento para todas las noticias del csv
df['sentimiento'] = df['texto'].fillna("").apply(clasificar_sentimiento)

# Gráfico de barras con distribución general
def graficar_distribucion_sentimientos():
    conteo = df['sentimiento'].value_counts()
    plt.figure(figsize=(6,4))
    conteo.plot(kind='bar', color=['green','red','gray'])
    plt.title("Distribución de sentimiento en artículos")
    plt.ylabel("Cantidad de artículos")
    plt.xlabel("Sentimiento")
    plt.xticks(rotation=0)
    plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close()
    buf.seek(0)
    return Image.open(buf)

# Mostrar sentimiento y gráfica de distribución al seleccionar artículo
def analizar_sentimiento(titulo_articulo):
    fila = df[df["titulo"].astype(str) == str(titulo_articulo)]
    if fila.empty:
        return "No se encontró el artículo.", None
    texto = fila.iloc[0]["texto"]
    sentimiento_articulo = clasificar_sentimiento(texto)  # Analiza solo este texto
    grafico = graficar_distribucion_sentimientos()
    texto_salida = f"Sentimiento del artículo seleccionado: **{sentimiento_articulo}**"
    return texto_salida, grafico




# 4. Función de resumen y generación de tweet

# Función para generar resumen
def generar_resumen(texto):
    pregunta = f"""Sumariza el siguiente texto en tres oraciones seguidas relacionadas de rapida lectura
              Texto: {texto}
              """

    resumen_final = cliente.models.generate_content(
                    model=MODEL_ID,
                    contents=[pregunta] # Pasa la pregunta como contenido
                    )
    return resumen_final.text


# Función para generar tweet
def generar_tweet(texto):
    pregunta = f"""Sumariza en un tweet de 80 palabras máximo utilizando modismos argentinos, no es necesario que remarques los modismos
              Texto: {texto}
              """

    tweet_final = cliente.models.generate_content(
                    model=MODEL_ID,
                    contents=[pregunta] # Pasa la pregunta como contenido
                    )
    return tweet_final.text



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 4. Interfaz Gradio

In [34]:
# --- INTERFAZ GRADIO ---

with gr.Blocks() as interfaz:
    gr.Markdown("## 📰 Visualizador")

    dropdown = gr.Dropdown(choices=opciones, label="Seleccioná un artículo")

    with gr.Row():
        boton_texto = gr.Button("Mostrar texto")
        boton_limpieza = gr.Button("Limpiar texto")
        boton_wc = gr.Button("Generar WordCloud")
        boton_ner = gr.Button("Extraer Entidades (NER)")
        boton_sent = gr.Button("Análisis de Sentimiento")

    with gr.Row():
        boton_resumen = gr.Button("Resumen")
        boton_opinion = gr.Button("Generador de Tweet")

    salida = gr.Textbox(label="Texto del artículo", lines=15)
    salida_texto = gr.Textbox(label="Limpieza", lines=15)
    salida_imagen = gr.Image(type="pil", label="WordCloud")
    salida_ner = gr.Textbox(label="Entidades nombradas extraídas")
    salida_sentimiento = gr.Markdown(label="Resultado de análisis de sentimiento")
    salida_grafico_sent = gr.Image(type="pil", label="Distribución general sentimientos del conjunto de noticias")
    salida_resumen = gr.Textbox(label="Resumen de la noticia", lines=3)
    salida_opinion = gr.Textbox(label="Generador de Tweet", lines=4)


    # Eventos botones
    boton_texto.click(fn=mostrar_texto, inputs=dropdown, outputs=salida)
    boton_limpieza.click(fn=procesar_texto, inputs=salida, outputs=salida_texto)
    boton_wc.click(fn=generar_wordcloud, inputs=salida_texto, outputs=salida_imagen)
    boton_ner.click(fn=realizar_ner, inputs=salida_texto, outputs=salida_ner)
    boton_sent.click(fn=analizar_sentimiento, inputs=dropdown, outputs=[salida_sentimiento, salida_grafico_sent])
    boton_resumen.click(fn=generar_resumen, inputs=salida_texto, outputs=salida_resumen)
    boton_opinion.click(fn=generar_tweet, inputs=salida_texto, outputs=salida_opinion)


interfaz.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://988e7dab409dbcefdd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
